# Importacao das libs

**ATENCAO:** *UTILIZE PYTHON __3.11__ OU SUPERIOR!*

In [ ]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
import camelot
from datetime import datetime

# Puxa o endereco das tabelas

In [ ]:
# url da Anvisa
url = "https://www.gov.br/anvisa/pt-br/setorregulado/regularizacao/medicamentos/medicamentos-de-referencia/lista-de-medicamentos-de-referencia"

# instanciando o firefox
driver = webdriver.Firefox()
driver.get(url)

sleep(10)

# buscando a lista de links
xpath = "//div[@id='parent-fieldname-text']//ul"
element = driver.find_element(By.XPATH, xpath)
html_content = element.get_attribute('outerHTML')
soup = BeautifulSoup(html_content, 'html.parser')

# separando os links
tableList = list()
for el in soup.find_all(name='a'):
    link = el.get('href')
    if 'https' in  link:
        tableList.append(link)

# encerra o firefox
driver.quit()


In [ ]:
# para conferir a lista com o endereco das tabelas
tableList

# Define as funcoes usados posteriormente

### funcao destinada a filtrar a tabela

In [ ]:
# filtro que funciona:
def filterTable(temp_table):
    for row in range(0, len(temp_table.df)):
        if temp_table.df.at[row, 3] != '' and '/' not in temp_table.df.at[row, 3]:
            for col in range(0, len(temp_table.df.columns)):
                if temp_table.df.at[row, col] == '':
                    # print(f'{temp_table.df.at[row-1, col]} {temp_table.df.at[row+1, col]}')
                    temp_table.df.at[row,col] = f'{temp_table.df.at[row-1, col]} {temp_table.df.at[row+1, col]}'

    for row in range(0, len(temp_table.df)):
        if temp_table.df.at[row, 3] == '' or '/' in temp_table.df.at[row, 3]:
            temp_table.df = temp_table.df.drop(index=row)
        
    temp_table.df = temp_table.df.reset_index(drop=True)
    
    return temp_table

In [ ]:
# filtro novo, nao funciona
def filterTable2(temp_table):
    for row in range(0, len(temp_table.df)):
        if temp_table.df.at[row, 3] == '':
            for col in range(0, len(temp_table.df.columns)):
                if temp_table.df.at[row, col] != '':
                    newData = list()
                    rw = 1
                    while True:
                        if temp_table.df.at[row+rw, 3] == '':
                            rw += 1
                        else:
                            break
                    for temp_row in range(0, rw+rw+1):
                        newData.append(temp_table.df.at[temp_row, col])
                        temp_table.df.at[temp_row, col] = ''
                    
                    print(' '.join(newData))
                    temp_table.df.at[row+rw, col] = ' '.join(newData)
    
    return temp_table

### funcao principal para analise das tabelas

In [ ]:
def getDatasOfTable(ntable):
    link_pdf = tableList[ntable]
    tables = camelot.read_pdf(link_pdf, flavor='stream', pages='all', flag='text')

    for pg in range(0, len(tables)):
        table = tables[pg]
        if pg == 0:
            match ntable:
                case 0:
                    table.df = table.df[2:].reset_index(drop=True)
                case 1:
                    table.df = table.df[4:].reset_index(drop=True)
                case 3:
                    table.df = table.df[4:].reset_index(drop=True)

        table = filterTable(table)
        tables[pg].df = table.df

    return tables


### funcao que gera um arquivo sql de exportacao

In [13]:
# declara o nome do arquivo de exportacao
dtn = datetime.now()
export_file_name = f'{dtn.day}-{dtn.month}-{dtn.year}-{dtn.hour}-{dtn.minute}.sql'

export_file_name

'22-3-2024-21-19.sql'

In [14]:
def exportSQL(temp_table, status):
    try:
        file = open('exports/'+export_file_name, 'r', encoding='utf-8')
        cmd = ''.join(file.readlines())
        file.close()
    except:
        cmd = ''

    file = open('exports/'+export_file_name, 'w', encoding='utf-8')

    cmd += f"INSERT INTO `Medicamentos` (farmaco, detentor, medicamento, concentracao, status) VALUES\n"

    for row in range(0, len(temp_table.df)):
        if "'" in temp_table.df.at[row, 0]:
            farmaco = temp_table.df.at[row, 0].replace("'", "")
        else:
            farmaco = temp_table.df.at[row, 0]

        if "'" in temp_table.df.at[row, 1]:
            detentor = temp_table.df.at[row, 1].replace("'", "")
        else:
            detentor = temp_table.df.at[row, 1]

        if "'" in temp_table.df.at[row, 2]:
            medicamento = temp_table.df.at[row, 2].replace("'", "")
        else:
            medicamento = temp_table.df.at[row, 2]

        if "'" in temp_table.df.at[row, 4]:
            concentracao = temp_table.df.at[row, 4].replace("'", "")
        else:
            concentracao = temp_table.df.at[row, 4]

        cmd0 = f" ('{farmaco}', '{detentor}', '{medicamento}', '{concentracao}', '{status}')"
        cmd += cmd0
        if row < len(temp_table.df)-1:
            cmd += ',\n'
        else:
            cmd += ';\n\n'
    
    file.write(str(cmd))
    file.close()

# Leitura das tabelas

In [ ]:
# declara uma tabela
# tabela A incluidos
myTable = getDatasOfTable(0)

In [ ]:
# exibi uma das paginas da tabela
display(myTable[10].df)

In [ ]:
exportSQL(myTable[10], 'inativo')

In [ ]:
for i in range(0, 2):
    if  i == 0:
        sts = 'ativo'
    else:
        sts = 'inativo'

    myTable = getDatasOfTable(i)
    for p in  myTable:
        exportSQL(p, sts)